In [1]:
import copy
import random
import pandas as pd

# Battleship

Rules:
1. Each player arranges ships according to fleet
2. Take turns firing a shot
3. Mark Hits and Misses
4. Call out when a ship has been sunk
5. Sink all to win

Ships:
1. Carrier - 5
2. Battleship - 4
3. Cruiser - 3
4. Submarine - 3
5. Destroyer -2

10 Rows x 10 Columns  

|   |   |   |   |   |   |   |   |   |   |
|---|---|---|---|---|---|---|---|---|---|
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |

##### Parameters

In [2]:
Enemy_Ships = [["Carrier", 5, []],["Battleship", 4, []],["Cruiser", 3, []],["Submarine", 3, []],["Destroyer", 2, []]]

##### Helper Functions

In [3]:
# Is current proposed ship allocation valid?

def ValidPlacement(Board, i, j, k, length):
    # Board = Dictionary for Board
    # i = row of ship
    # j = column of ship
    # k = orientation of ship (0 = hoz, 1 = vert)
    # length = length of ship
    
    #Horizontal Ship
    if (k == 0):
        if (j>11-length):
            return 0
        else:
            occupied = 0
            for l in range(length):
                occupied = occupied + Board[i][j+l]
            if(occupied == 0):
                return 1
            else:
                return 0
            
    #Vertical Ship
    if (k == 1):
        if (i>11-length):
            return 0
        else:
            occupied = 0
            for l in range(length):
                occupied = occupied + Board[i+l][j]
            if(occupied == 0):
                return 1
            else:
                return 0

In [4]:
def ValidLocations(Board, Ship_List):
    #Wri
    #Board: dictionary for occupied spaces
    #Ship_List: Array with ship name, length, 3rd index to place valid locations 
    #          ["Ship_Name", length, []]
    
    for s in range(len(Ship_List)):
        valid_placements = []
        Ship = Ship_List[s]
        length = Ship[1]

        for i in range(1,11):
            for j in range(1,11):
                for k in range(2):
                     if (ValidPlacement(Board,i,j,k,length)==1):
                            valid_placements.append([i,j,k])
        Ship[2] = valid_placements

In [5]:
def SpacesOccupied(length,location):
    #length = length of ship
    #location = [i,j,k]
    #Returns list of spaces occupied by ship
    
    occupied = []
    i,j,k = location
    
    if (k==0):
        for l in range(length):
            occupied.append((i,j+l))
    if (k==1):
        for l in range(length):
            occupied.append((i+l,j))
        
    return occupied

In [6]:
def RandomSample(Ship_List):
    # Ship_List = ["Ship_Name", length, locations]
    # Return: Data frame of board with 1s for ship positions
    #        or all 0s if not a valid arrangement
    
    PDF = pd.DataFrame(index=range(1,11), columns=range(1,11))
    PDF = PDF.fillna(0) # with 0s rather than NaNs
    
    locations = []
    for s in range(len(Ship_List)):
        Ship = Ship_List[s]
        position = random.choice(Ship[2])
        spaces = SpacesOccupied(Ship[1],position)
        locations.extend(spaces)
    #Check if valid configuration
    if(len(locations) == len(set(locations))):
        for t in locations:
            PDF.loc[t] = 1
        return PDF
    else:
        return False        

In [18]:
def NSamples(N,M,Ship_List):
    #N = Number of samples wanted
    #M = Max number of iterations
    #Ship_List = ["Ship_Name", length, locations]
    #Returns: (DF, S) Data frame of accumulated possible locations and number of actual samples taken
    
    PDF = pd.DataFrame(index=range(1,11), columns=range(1,11))
    PDF = PDF.fillna(0) # with 0s rather than NaNs
    n = 0
    m = 0
    while( (n<N) & (m<M) ):
        m = m + 1
        Result = RandomSample(Ship_List)
        if (type(Result) != bool):
            PDF = PDF + Result
            n = n+1
    
    return(PDF,n)

##### Initialize Board

In [58]:
Board = {}
for i in range(1,11):
    Board[i] = {}
    for j in range(1,11):
        Board[i][j] = 0 

##### Example

In [65]:
Board[5][5] = 1
Board[5][6] = 1
Board[6][5] = 1
Board[6][6] = 1

Board[2][2] = 1
Board[3][3] = 1
Board[4][4] = 1
Board[5][1] = 1

Board[10][10] = 1
Board[8][3] = 1
Board[2][7] = 1
Board[10][9] = 1

In [66]:
ValidLocations(Board, Enemy_Ships)

In [67]:
PDF,n = NSamples(2000,6000,Enemy_Ships)

In [68]:
PDF

,1,2,3,4,5,6,7,8,9,10
1,229,269,394,486,530,506,388,444,368,232
2,152,0,123,240,279,219,0,276,309,284
3,156,145,0,165,359,479,469,616,539,458
4,148,309,132,0,208,291,474,555,546,474
5,0,386,214,168,0,0,375,448,530,500
6,218,527,281,307,0,0,401,441,477,432
7,333,594,389,644,456,465,652,547,497,407
8,312,361,0,375,301,403,564,539,443,336
9,381,476,331,597,512,502,550,501,348,245
10,288,338,362,536,490,425,347,197,0,0


In [69]:
PDF/n

,1,2,3,4,5,6,7,8,9,10
1,0.1145,0.1345,0.1970,0.2430,0.2650,0.2530,0.1940,0.2220,0.1840,0.1160
2,0.0760,0.0000,0.0615,0.1200,0.1395,0.1095,0.0000,0.1380,0.1545,0.1420
3,0.0780,0.0725,0.0000,0.0825,0.1795,0.2395,0.2345,0.3080,0.2695,0.2290
4,0.0740,0.1545,0.0660,0.0000,0.1040,0.1455,0.2370,0.2775,0.2730,0.2370
5,0.0000,0.1930,0.1070,0.0840,0.0000,0.0000,0.1875,0.2240,0.2650,0.2500
6,0.1090,0.2635,0.1405,0.1535,0.0000,0.0000,0.2005,0.2205,0.2385,0.2160
7,0.1665,0.2970,0.1945,0.3220,0.2280,0.2325,0.3260,0.2735,0.2485,0.2035
8,0.1560,0.1805,0.0000,0.1875,0.1505,0.2015,0.2820,0.2695,0.2215,0.1680
9,0.1905,0.2380,0.1655,0.2985,0.2560,0.2510,0.2750,0.2505,0.1740,0.1225
10,0.1440,0.1690,0.1810,0.2680,0.2450,0.2125,0.1735,0.0985,0.0000,0.0000


In [70]:
n

2000

### To Do

1. Code should be divided into "Ship Search" and "Ship Sink" methods
    * Ship Search - tries to find ship to sink (mostly written)
    * Ship Sink - tries to sink ship when one is found (need to write)
2. Find space with highest probability to shoot into for "Ship Search"
3. Write "Ship Sink" method
    * If enemy ship is hit, iterate through list of possible ships that were hit to find most likely direction to start testing
    * Edge cases very important. Possible to hit multiple ships in trying tosink one so "ship sunk" queue very important
    